Oulu_NLPTM_TwitterBrexit

In [8]:
import os
import tweepy as tw
import pandas as pd

import sys
sys.path.insert(0, '../config')

from twitter_dev_access import *


In [10]:
# access twitter
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)


In [27]:

tweets = tw.Cursor(api.user_timeline, screen_name='@realDonaldTrump', tweet_mode="extended").items(1000)

for i, tweet in enumerate(tweets):
    print("Tweet " + str(i) + ": " + tweet.full_text)


Tweet 0: Sen. Ron Johnson Calls For FBI To Address Reported Biden Ukraine Meeting https://t.co/UGVw1gEW7T via @BreitbartNews
Tweet 1: Watch the @marklevinshow on @FoxNews at 10:00 P.M. He will importantly be talking about the corruption of Joe Biden. @SteveHiltonx and Larry Kudlow were great tonight!
Tweet 2: https://t.co/ResTvtIIr5
Tweet 3: THANK YOU NEVADA! https://t.co/WCfXKZ1Vpu
Tweet 4: https://t.co/swBqIhNVc9
Tweet 5: https://t.co/Oay4jTvbUL
Tweet 6: https://t.co/mgmCZj5jgh
Tweet 7: https://t.co/yEbGpzVWhH
Tweet 8: Big show tonight on @FoxNews! @MarkLevinShow will be exposing Joe Biden’s Corruption at 8:00pmE — and of course, don’t miss @NextRevFNC with @SteveHiltonx at 9:00pmE!
Tweet 9: https://t.co/BK010CENQO
Tweet 10: https://t.co/vOw2bXAtBT
Tweet 11: THANK YOU Newport Beach, California! Now on my way to Carson City, Nevada. Another big crowd, see you soon! #MAGA https://t.co/QV8V9SdzJI
Tweet 12: Corrupt politician Joe Biden makes Crooked Hillary look like an amateur!
Tweet 13

Tweet 80: https://t.co/2mkecBJH46
Tweet 81: https://t.co/t7MUFQi3G2
Tweet 82: RT @DanScavino: 🚨HAPPENING NOW‼️
10/16/20–Macon, GA! MAGA🇺🇸🦅
🌐https://t.co/zqgeZML9Hf https://t.co/pY20ZQ6sUz
Tweet 83: THANK YOU MACON, GEORGIA!
https://t.co/jjnVrVgQWY
Tweet 84: Getting ready to land in Macon, Georgia. Another giant crowd. See you soon!
Tweet 85: https://t.co/SpBa6xZorz
Tweet 86: https://t.co/74sqFSP4o3
Tweet 87: RT @TeamTrump: President @realDonaldTrump: Early voting in #Florida starts on MONDAY, and we are going to have a Red Wave!

VISIT: https://…
Tweet 88: THANK YOU OCALA, FLORIDA!
https://t.co/w6KmVYR6kA
Tweet 89: Getting ready to speak in Florida. Enjoy!
Tweet 90: RT @GOP: The numbers speak for themselves! Pres. Trump has provided health care SAVINGS to our seniors.

AZ—premiums ⬇️22% after ⬆️64% w/ O…
Tweet 91: RT @GOP: “We love our senior citizens…You devoted your life to this country and I am devoting my life to you.”—@realDonaldTrump https://t.c…
Tweet 92: RT @TeamTrump: Presiden

Tweet 160: RT @jasoninthehouse: Sickening
Tweet 161: RT @jasoninthehouse: Obama conference call leaked to Burisma: Biden emails https://t.co/ckNMurwMq0
Tweet 162: THANK YOU NORTH CAROLINA! https://t.co/4epZQPjfi7
Tweet 163: Pelosi is holding up STIMULUS, not the Republicans!
Tweet 164: I will be doing a major Fake @NBCNews Town Hall Forum, live tonight from Miami, at 8:00 P.M. They asked me to do it in place of the Rigged Steve Scully (he is now suspended from @cspan for lying) Debate. I wonder if they’ll treat me as well as Sleepy Joe? They should!
Tweet 165: I was right again! Steve Scully just admitted he was lying about his Twitter being hacked. The Debate was Rigged! He was suspended from @cspan indefinitely. The Trump Campaign was not treated fairly by the “Commission”. Did I show good instincts in being the first to know?
Tweet 166: If we don’t win, the Radical Left will destroy our Country. Biden refuses to answer questions on packing the SCOTUS! Dems will pack the Court w/ rad

In [ ]:
# testAPI
search_words = "#wildfires"
date_since = "2018-11-16"


# Collect tweets
tweets = tw.Cursor(api.search, q=search_words, lang="en", since=date_since).items(5)

for i, tweet in enumerate(tweets):
    print("Tweet " + str(i) + ": " + tweet.text)

In [36]:
pages = tw.Cursor(api.user_timeline, id='id', count=200).pages()

print(pages.length)
for page in pages:
    for tweet in page:
        print(tweet.text)

AttributeError: 'IdIterator' object has no attribute 'length'